In [1]:
#@title Installs
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet #8
!pip install tensorflow-text==2.15.0 --quiet #15
#!pip install transformers==4.17 --quiet #4.40.2 #4.37.2
!pip install transformers==4.29.0
!pip install simpletransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
#@title Imports

import torch
import transformers
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import random
from sklearn.metrics import accuracy_score, f1_score

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
import transformers

from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import RobertaTokenizer, TFRobertaModel

from transformers import logging
logging.set_verbosity_error()

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from simpletransformers.classification import MultiLabelClassificationModel

In [3]:
#@title Utility print function

def print_version(library_name):
    try:
        lib = __import__(library_name)
        version = getattr(lib, '__version__', 'Version number not found')
        print(f"{library_name} version: {version}")
    except ImportError:
        print(f"{library_name} not installed.")
    except Exception as e:
        print(f"An error occurred: {e}")

#confirm versions
print_version('numpy')
print_version('transformers')
print_version('tensorflow')
print_version('keras')
print_version('pandas')
print_version('sklearn')

numpy version: 1.25.2
transformers version: 4.43.2
tensorflow version: 2.15.0
keras version: 2.15.0
pandas version: 2.0.3
sklearn version: 1.2.2


In [4]:
#@title Global tunable parameters

# Sequence length to truncate/pad
MAX_SEQUENCE_LENGTH = 1000

# hidden layer size after BERT's ouput
HIDDEN_LAYER_SIZE = 512

In [5]:
#@title Utility Plot Function

# 4-window plot. Small modification from matplotlib examples.

def make_plot(axs,
              model_history1,
              model_history2,
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        # small adjustment to account for the 2 accuracy measures in the Weighted Averging Model with Attention
        if 'classification_%s' % metric in model_history2.history:
            metric2 = 'classification_%s' % metric
        else:
            metric2 = metric

        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric2])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric2])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            model_metric = [metric, metric2][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[model_metric])
            ax1.plot(model_history.history['val_%s' % model_metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

In [6]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#@title Read Reddit dataset into a dataframe
rdt_trainfile = '/content/drive/MyDrive/MIDS/w266/02_final_project/mids-w266-project/Reddit/both_train.csv'
rdt_tesstfile = '/content/drive/MyDrive/MIDS/w266/02_final_project/mids-w266-project/Reddit/both_test.csv'
rdt_train = pd.read_csv(rdt_trainfile)
rdt_test = pd.read_csv(rdt_tesstfile)

# Shuffle all rows
rdt_train = rdt_train.sample(frac=1).reset_index(drop=True)
rdt_test = rdt_test.sample(frac=1).reset_index(drop=True)

train_labels = rdt_train.class_id
test_labels = rdt_test.class_id

temp_train_examples = rdt_train.post
temp_test_examples = rdt_test.post

train_examples_list = []
test_examples_list = []

for i in range(len(temp_train_examples)):
  size = len(temp_train_examples[i])
  lower = int(0.3*MAX_SEQUENCE_LENGTH)
  upper = int(0.7*MAX_SEQUENCE_LENGTH)
  if size > MAX_SEQUENCE_LENGTH:
    train_examples_list.append(temp_train_examples[i][:lower] + temp_train_examples[i][upper:])
  else:
    train_examples_list.append(temp_train_examples[i])

for i in range(len(temp_test_examples)):
  size = len(temp_test_examples[i])
  lower = int(0.3*size)
  upper = int(0.7*size)
  if size > MAX_SEQUENCE_LENGTH:
    test_examples_list.append(temp_test_examples[i][:lower] + temp_test_examples[i][upper:])
  else:
    test_examples_list.append(temp_test_examples[i])

train_examples = np.array(train_examples_list)
test_examples = np.array(test_examples_list)


In [8]:
df1 = pd.read_csv("/content/drive/MyDrive/MIDS/w266/02_final_project/mids-w266-project/Reddit/both_train.csv")
df2 = pd.read_csv("/content/drive/MyDrive/MIDS/w266/02_final_project/mids-w266-project/Reddit/both_test.csv")

In [9]:
df1

,ID,title,post,class_name,class_id
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5
...,...,...,...,...,...
13722,386a4117-0324-4b25-8330-b1a6857ccaa6,trying to wake up and leave the house every mo...,"i don't often succeed and when i do, it's like...",bipolar,2
13723,4dd92d63-1231-410d-b049-5d8c430c2f36,"""hey man, what'd you do this past weekend?""",fuck i hate this question. fuck off. i got hom...,depression,3
13724,dbde1d3e-e527-4cf8-8045-2093aec2a784,"how to use "" whereby "" correctly?",hi. i'm not a native speaker and not a good en...,none,5
13725,5355fad6-5bb4-4092-8428-9b8527a6d261,"i'm sorry, another depression post",i'm sorry to be such a downer all the time. i ...,bipolar,2


In [10]:
df2

,ID,title,post,class_name,class_id
0,b07b978c-7760-4932-85cb-5797b5e74168,"basic needs for neurotypicals : food, clothing...",most people able to be productive and function...,adhd,0
1,b2868fd2-e885-400e-b780-446c9581b1f1,"""you're just an iphone in an android world.: d...",i'm 23f and recently quit my job as a features...,adhd,0
2,3191b3b1-d355-41b6-9588-caeb3e169c9b,"""what kind of music do you listen to?"" no answer",i just got finished with an interview and as t...,adhd,0
3,a3d60895-ef7c-4b57-b5be-632c9fd62150,couldn’t help crying while reading a post that...,off course i’ve forgotten what i exactly read....,adhd,0
4,66fae18e-e744-495f-b7a0-b985543f5052,shout-out to all the parents with adhd childre...,"i have six siblings, and at least 4 of us have...",adhd,0
...,...,...,...,...,...
1483,5855d35b-bf7d-4a49-8030-b185f430f05a,i went on a solo motorcycle ride from tamil na...,i quit my job early this year and moved back t...,none,5
1484,a2faea38-8b93-43da-a75b-cb7c0ba1ae0d,hello and namaste to r/india... from r/nepal. ...,there are lots of trolls trying to create a di...,none,5
1485,7ad72b7f-eba4-4c17-90b0-1038dc144a8b,public api for the largest video game database...,&amp;#x200b; rawg is the largest video game da...,none,5
1486,e73c198f-5e8e-4c84-aedf-333b576f9939,acs ama: hi reddit! my name is dr. john m news...,hi reddit! i’m [dr. john m. and i serve as ceo...,none,5


In [11]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [12]:
corpus["text"] = corpus.title + " " + corpus.post

In [13]:
corpus

,ID,title,post,class_name,class_id,text
0,691324c4-5c30-44e0-b9e4-45b4f0715e21,a question about the third conditional.,i was making questions for my students and i r...,none,5,a question about the third conditional. i was ...
1,d4295391-9ca5-4398-b7c8-687e4a984ef1,the epitome of my life,i've recently requested testing accommodations...,adhd,0,the epitome of my life i've recently requested...
2,58937fa5-3c2c-426b-8255-5a140fbab675,what are your favourites offbeat destinations ...,**cambodia** * koh rong: amazing beaches and a...,none,5,what are your favourites offbeat destinations ...
3,7daf364c-3b33-4cbe-be37-a214edf9a73e,synesthesia survey (what colour is each month ...,synesthesia. what is synesthesia? according to...,none,5,synesthesia survey (what colour is each month ...
4,22518271-4bb4-4caf-b683-7305da519288,"science ama series: i’m phil baran, and i’m he...",i’m phil baran and i teach organic chemistry a...,none,5,"science ama series: i’m phil baran, and i’m he..."
...,...,...,...,...,...,...
15210,5855d35b-bf7d-4a49-8030-b185f430f05a,i went on a solo motorcycle ride from tamil na...,i quit my job early this year and moved back t...,none,5,i went on a solo motorcycle ride from tamil na...
15211,a2faea38-8b93-43da-a75b-cb7c0ba1ae0d,hello and namaste to r/india... from r/nepal. ...,there are lots of trolls trying to create a di...,none,5,hello and namaste to r/india... from r/nepal. ...
15212,7ad72b7f-eba4-4c17-90b0-1038dc144a8b,public api for the largest video game database...,&amp;#x200b; rawg is the largest video game da...,none,5,public api for the largest video game database...
15213,e73c198f-5e8e-4c84-aedf-333b576f9939,acs ama: hi reddit! my name is dr. john m news...,hi reddit! i’m [dr. john m. and i serve as ceo...,none,5,acs ama: hi reddit! my name is dr. john m news...


In [14]:
corpus[["class_name", "class_id"]].value_counts()

class_name  class_id
adhd        0           2713
depression  3           2698
anxiety     1           2670
bipolar     2           2655
ptsd        4           2249
none        5           2230
Name: count, dtype: int64

In [15]:
# if torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")
# torch.cuda.get_device_name(0)

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.get_device_name(0) # Only get device name if CUDA is available
else:
    device = torch.device("cpu")

In [16]:
robertaCorpus = corpus[["text","class_id"]]
robertaCorpus

,text,class_id
0,a question about the third conditional. i was ...,5
1,the epitome of my life i've recently requested...,0
2,what are your favourites offbeat destinations ...,5
3,synesthesia survey (what colour is each month ...,5
4,"science ama series: i’m phil baran, and i’m he...",5
...,...,...
15210,i went on a solo motorcycle ride from tamil na...,5
15211,hello and namaste to r/india... from r/nepal. ...,5
15212,public api for the largest video game database...,5
15213,acs ama: hi reddit! my name is dr. john m news...,5


In [17]:
robertaCorpus["class_id"].value_counts()

class_id
0    2713
3    2698
1    2670
2    2655
4    2249
5    2230
Name: count, dtype: int64

In [18]:
# from simpletransformers.classification import MultiLabelClassificationModel
!pip install --upgrade transformers
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=10,
    use_cuda=False,# Set to True if using a GPU
    attn_implementation="eager" # Add this line to force eager mode
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [19]:
from simpletransformers.model import TransformerModel
model = TransformerModel('roberta', 'roberta-base', num_labels=10, args={'learning_rate':1e-5, 'num_train_epochs': 2, 'reprocess_input_data': True, 'overwrite_output_dir': True})
# model = TransformerModel('roberta', 'roberta-base', num_labels=10, use_cuda=False, args={'learning_rate':1e-5, 'num_train_epochs': 2, 'reprocess_input_data': True, 'overwrite_output_dir': True})

In [20]:
# from sklearn.model_selection import train_test_split

train, test = train_test_split(robertaCorpus, test_size=0.2)

In [21]:
model.train_model(train)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/24 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1522 [00:00<?, ?it/s]

Running Epoch 2 of 2:   0%|          | 0/1522 [00:00<?, ?it/s]

(3044, 0.6308558073369651)

In [22]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/6 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/31 [00:00<?, ?it/s]

In [23]:

print(result)
print(model_outputs)
print(wrong_predictions)

{'mcc': 0.824471420274575, 'f1': 0.8534341110745974, 'acc': 0.8534341110745974, 'eval_loss': 0.4849561337501772}
[[-0.6953125   0.31738281  0.65722656 ... -1.85449219 -1.56347656
  -1.90039062]
 [ 5.4453125   0.05197144  2.09765625 ... -1.47070312 -1.42285156
  -1.50097656]
 [ 1.21972656  0.68017578  2.52148438 ... -1.58496094 -1.90136719
  -1.77148438]
 ...
 [-0.42431641  0.33154297  5.859375   ... -1.69042969 -1.65527344
  -1.76855469]
 [ 6.703125   -0.61035156 -0.54443359 ... -0.95410156 -1.03515625
  -0.74658203]
 [-0.828125    1.13476562  1.36425781 ... -2.01171875 -1.70898438
  -2.00585938]]
[{'guid': 8, 'text_a': 'to those who ask "are you ok?" are you ok can have a different meaning depending on who it\'s coming from. there\'s a huge gray area when it comes to answering that question. they could be wondering if you\'re feeling a little down. they could be asking if you\'re having a really rough time. they could be asking if you\'re in control. they could be asking if you\'re go